In [1]:
import os
current_path = os.path.dirname(os.path.abspath(__name__))
folder_path = os.path.join(current_path,'每日各站進出站人數')
fileName_list = []
for filename in os.listdir(folder_path):
    if '每日各站進出站人數' in filename:
        fileName_list.append(os.path.join(folder_path,filename))

In [2]:
print(fileName_list)

['/workspaces/__2024_07_01_Shilin_python__/lesson19/每日各站進出站人數/每日各站進出站人數2020.csv', '/workspaces/__2024_07_01_Shilin_python__/lesson19/每日各站進出站人數/每日各站進出站人數2021.csv', '/workspaces/__2024_07_01_Shilin_python__/lesson19/每日各站進出站人數/每日各站進出站人數20190423-20191231.csv', '/workspaces/__2024_07_01_Shilin_python__/lesson19/每日各站進出站人數/每日各站進出站人數2023.csv', '/workspaces/__2024_07_01_Shilin_python__/lesson19/每日各站進出站人數/每日各站進出站人數2022.csv']
